In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
import glob
import nibabel as nib


class BratsDataset(Dataset):

    def __init__(self, folder_dir):

        self.folder_dir = folder_dir
        self.image_files = sorted(glob.glob((os.path.join(self.folder_dir, '**/*flair.nii.gz')), recursive = True))
        self.mask_files = sorted(glob.glob((os.path.join(self.folder_dir, '**/*seg.nii.gz')), recursive= True))

    def __len__(self):

        return len(self.image_files)

    def __getitem__(self, idx):
        
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_files[idx])

        image = nib.load(img_path)
        mask = nib.load(mask_path)

        image = np.expand_dims(image, axis=0)
        mask = np.expand_dims(mask, axis=0)

        image = torch.from_numpy(image)
        mask = torch.from_numpy(mask)

        return image, mask
    

train_path = '/Users/daniel/Documents/CSAI/Advanced Deep Learning/CODE/data/train/'